# Thu thập dữ liệu

## Giới thiệu
Dự án xây dựng mô hình dự đoán giá ô tô thông qua dữ liệu thực tế lấy từ các website mua bán xe trực tuyến. Notebook này tóm tắt quy trình lấy dữ liệu từ hai trang mua bán lớn của Việt Nam nhằm chuẩn bị cho việc chuẩn hóa và mô hình hóa.

## Mục tiêu
- Thu thập các thông số kỹ thuật (features) chính xác từ mỗi website.
- Ghi nhận nguồn gốc dữ liệu để so sánh nhanh giữa hai trang.

## Phạm vi
- Nguồn: Chotot (API) và Bonbanh (web scraping).
- Dữ liệu: Tiêu đề, giá, thông số kỹ thuật, vị trí, người bán, số ảnh (nếu có).
- Kết quả: CSV trong `datasets/` phục vụ phân tích và huấn luyện mô hình.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from random import uniform
from concurrent.futures import ThreadPoolExecutor, as_completed

N = 10000  # Số lượng mẫu cần thu thập từ mỗi trang web
session = requests.Session() # Tạo một phiên riêng để tái sử dụng kết nối

## Phương pháp

### 1. Thu thập dữ liệu
- **Chotot**: Dùng API nội bộ để lấy ID và sinh URL rao bán xe. Sau đó lấy thông tin chi tiết của xe từ URL đã sinh.
- **Bonbanh**: Đọc trang `bonbanh.com/oto` với phân trang để scrape danh sách listing, sau đó truy cập từng URL chi tiết.

### 2. Tối ưu hiệu suất
- Sử dụng **đa luồng** (threading) để trích xuất thông tin từ nhiều URL song song, giảm thời gian chờ đợi.
- Mỗi luồng xử lý một URL độc lập, kết quả được gom lại sau khi tất cả luồng hoàn thành.

### 3. Trích xuất features
#### I.Chotot
- Lấy html của trang bán xe chi tiết
- Tiêu đề/giá nằm ở các tag rõ ràng (`<h1>` và `<b class="p26z2wb">`).
- Tất cả features được gom trong `div.p1ja3eq0`; cặp `span.bwq0cbs` đầu là label, sau là giá trị.
- Đọc vị trí, người bán, mô tả để đánh dấu dữ liệu có đủ thông tin.

In [ ]:
def extract_chotot_features(url):
    """
    Trích xuất features chi tiết từ trang listing Chotot.
    Trả về dict với các label rõ ràng để merge sau này.
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        features = {}

        # Tiêu đề
        title_elem = soup.find('h1')
        if title_elem:
            features['title'] = title_elem.get_text(strip=True)

        # Giá
        price_elem = soup.find('b', class_='p26z2wb')
        if price_elem:
            features['price'] = price_elem.get_text(strip=True)

        # Features từ div.p1ja3eq0 (label trước, value sau)
        feature_divs = soup.find_all('div', class_='p1ja3eq0')
        for div in feature_divs:
            spans = div.find_all('span', class_='bwq0cbs')
            if len(spans) >= 2:
                key = spans[0].get_text(strip=True)
                value = spans[1].get_text(strip=True)
                if key and value:
                    features[key] = value

        # Vị trí
        location_elem = soup.find('span', class_='bwq0cbs', string=re.compile(r'Quận|Phường|Huyện|Tp|Tỉnh'))
        if location_elem:
            features['location'] = location_elem.get_text(strip=True)

        # Người bán
        seller_link = soup.find('a', href=re.compile(r'/user/'))
        if seller_link:
            seller_name = seller_link.find('b')
            if seller_name:
                features['seller'] = seller_name.get_text(strip=True)

        # Mô tả
        desc_elem = soup.find('div', class_='des_txt')
        if desc_elem:
            features['description'] = desc_elem.get_text(strip=True)

        return features
    except Exception as e:
        print(f"Lỗi khi trích xuất từ {url}: {e}")
        return {}


#### II.Bonbanh
- Sử dụng header của điện thoại vì có cấu trúc đơn giản và có thể lấy features dễ hơn máy tính
- Lấy html của trang bán xe chi tiết
- Tiêu đề xuất hiện trong `<h1>`.
- Tên hãng, tên xe nằm trong tiêu đề.
- Giá xe được trích xuất từ `div.price_box`
- Các thông số kỹ thuật được phân thành `div.row` (các dòng), mỗi cặp `<label>` và `<span class="inp">` tương ứng với tên và giá trị của thuộc tính.
- Lọc chỉ giữ những thuộc tính cốt lõi như Năm sản xuất, Tình trạng, Hộp số... để bảng dữ liệu không bị loãng với thông tin thừa.

In [ ]:
def extract_bonbanh_features(url, retry_count=3):
    """
    Trích xuất features từ URL Bonbanh.
    """
    for attempt in range(retry_count + 1):
        try:
            headers = {
                  'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1',
                  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
                  'Referer': 'https://www.google.com/'
            }
            response = session.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            features = {}

            # 1. Tiêu đề
            title_elem = soup.find('h1')
            if title_elem:
                features['title'] = title_elem.get_text(strip=True)

            # 2. Giá bán 
            # Tìm thẻ div.price_box -> h3 -> b
            price_box = soup.find('div', class_='price_box')
            if price_box:
                price_elem = price_box.find('b')
                if price_elem:
                    features['price'] = price_elem.get_text(strip=True)

            # 3. Các thông số kỹ thuật
            # Mapping từ Label trong HTML -> Key mong muốn trong features
            label_mapping = {
                'Năm SX': 'Năm sản xuất',
                'Tình trạng': 'Tình trạng',
                'Đã đi': 'Số Km đã đi',
                'Xuất xứ': 'Xuất xứ',
                'Kiểu dáng': 'Kiểu dáng',
                'Hộp số': 'Hộp số',
                'Động cơ': 'Động cơ',
                'Màu xe': 'Màu ngoại thất',
                'Nội thất': 'Màu nội thất',
                'Chỗ ngồi': 'Số chỗ ngồi',
                'Số cửa': 'Số cửa',
                'Dẫn động': 'Dẫn động'
            }

            # Tìm tất cả các hàng chứa thông tin (div.row trong #detail_box_1)
            rows = soup.find_all('div', class_='row')

            for row in rows:
                # Mỗi row có thể chứa 1 hoặc 2 cột (col, col2)
                cols = row.find_all('div', class_=lambda x: x and ('col' in x))

                for col in cols:
                    label_elem = col.find('label')
                    inp_elem = col.find('span', class_='inp')

                    if label_elem and inp_elem:
                        raw_label = label_elem.get_text(strip=True).replace(':', '')
                        value = inp_elem.get_text(strip=True)

                        if raw_label in label_mapping:
                            standard_key = label_mapping[raw_label]
                            features[standard_key] = value

            return features

        except Exception as e:
            if attempt < retry_count:
                print(f"Lỗi khi trích xuất từ {url} (lần thử {attempt + 1}): {e}. Thử lại...")
                sleep(1) 
            else:
                print(f"Lỗi khi trích xuất từ {url} sau {retry_count + 1} lần thử: {e}")
                return {}

### 3. Triển khai Thu thập URL
Trước khi trích xuất feature, chúng ta cần danh sách URL cụ thể. Chotot cung cấp API giúp tránh việc iterate qua từng trang tìm kiếm; Bonbanh không có API công khai nên phải scrape danh sách car-item.

#### Thu thập URL từ Chotot và Bonbanh
- Chotot: gọi API `gateway.chotot.com/v1/public/ad-listing` với tham số `limit` và sử dụng pagination để lấy `list_id`. Ghép thành URL đầy đủ từ id vừa lấy. Lặp qua các trang cho đến khi đủ n URL hoặc hết dữ liệu.

In [ ]:
def collect_chotot_urls(n=10):
    """
    Thu thập N URL đầu từ trang mua bán ô tô trên chotot.com
    bằng cách gọi API gateway với pagination.
    """
    base_url = "https://gateway.chotot.com/v1/public/ad-listing"
    urls = []
    page = 0
    while len(urls) < n:
          params = {
              'w': 1,          # web flag
              'limit': min(50, n - len(urls)),  # max 50 per request
              'st': 's,k',     # sort by score, keyword
              'f': 'p',        # filter published
              'cg': 2010,      # category: mua bán ô tô
              'o': page * 50,  # offset
              'page': page     # page
          }
          print(f"Processing page {page}")
          try:
              response = requests.get(base_url, params=params, timeout=20)
              response.raise_for_status()
              data = response.json()

              ads = data.get('ads', [])
              if not ads:
                  break  # no more ads

              for ad in ads:
                  list_id = ad.get('list_id')
                  if list_id:
                      url = f"https://xe.chotot.com/mua-ban-oto/{list_id}.htm"
                      urls.append(url)
                      if len(urls) >= n:
                          break

              page += 1
          except requests.RequestException as e:
              print(f"Lỗi khi gọi API trang {page}: {e}")
              break

    return urls[:n]

# Thu thập N URL đầu
chotot_urls = collect_chotot_urls(N)
print(f"Đã thu thập {len(chotot_urls)} URL từ Chotot:")
for url in chotot_urls:
    print(url)

- Bonbanh: dùng header của điện thoại để trang web có cấu trúc đơn giản, dễ lấy thông tin và hạn chế bị server nghi ngờ. Scrape trang `bonbanh.com/oto/page,{page}` để lấy `href` từ `a.top` (chứa link bán xe chi tiết). Lặp qua các trang cho đến khi đủ n URL hoặc hết listing. 

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import uniform

def collect_bonbanh_urls(n=10, retry_count=3):
    """
    Thu thập N URL đầu từ trang mua bán ô tô cũ trên bonbanh.com
    bằng cách scrape trang tìm kiếm với nhiều trang.
    """
    urls = []
    page = 1
    while len(urls) < n:
        for attempt in range(retry_count + 1):
            try:
                url = f"https://bonbanh.com/oto/page,{page}"
                headers = {
                    'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1',
                    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
                    'Referer': 'https://www.google.com/'
                }
                response = requests.get(url, headers=headers, timeout=10)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
                print(f"processing page {page}")

                # Tìm trực tiếp thẻ a có class="top"
                a_tags = soup.find_all('a', class_='top', href=True)

                if not a_tags:
                    # Nếu trang tải ok mà không tìm thấy thẻ nào -> có thể hết xe hoặc bị chặn
                    return urls

                for a_tag in a_tags:
                    href = a_tag['href']

                    if href not in urls:
                        urls.append(href)
                        if len(urls) >= n:
                            return urls

                page += 1
                break  # Thành công
            except requests.RequestException as e:
                if attempt < retry_count:
                    print(f"Lỗi khi scrape trang {page} (lần thử {attempt + 1}): {e}. Thử lại...")
                    sleep(1) 
                else:
                    print(f"Lỗi khi scrape trang {page} sau {retry_count + 1} lần thử: {e}. Bỏ qua trang này.")
                    page += 1
                    break

    return urls[:n]

bonbanh_urls = collect_bonbanh_urls(N)
print(f"Đã thu thập {len(bonbanh_urls)} URL từ Bonbanh:")
for url in bonbanh_urls:
    print(url)


### 4. Triển khai Trích xuất và Lưu Dữ liệu với Đa luồng
Thay vì trích xuất tuần tự từng URL (chậm vì phải chờ mỗi request), ta dùng `ThreadPoolExecutor` để chạy song song nhiều request cùng lúc. Mỗi luồng gọi hàm trích xuất cho một URL, kết quả được gom lại khi tất cả luồng hoàn thành. Điều này giảm đáng kể thời gian xử lý khi có nhiều URL.

- Chotot: Không khắt khe trong việc có nhiều requests gửi tới cùng lúc từ 1 IP, có thể sử dụng số thread lớn và không cần delay
- BonBanh: Cực nhạy trong việc phát hiện có nhiều request gửi tới cùng lúc từ 1 IP, cần chỉnh số thread nhỏ và thêm delay

In [ ]:
# I. Trích xuất dữ liệu từ Chotot
print("Bắt đầu trích xuất dữ liệu Chotot...")
chotot_data = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(extract_chotot_features, url): url for url in chotot_urls}
    for future in as_completed(futures):
        url = futures[future]
        try:
            features = future.result()
            if features:
                print(features)
                chotot_data.append(features)
        except Exception as e:
            print(f"Lỗi với URL Chotot {url}: {e}")

df_chotot = pd.DataFrame(chotot_data)
df_chotot.to_csv('MyDrive/raw_chotot_car_features.csv', index=False)
print(f"Chotot: Đã lưu {len(chotot_data)} bản ghi vào datasets/raw_chotot_car_features.csv")

# II. Trích xuất dữ liệu từ Bonbanh
print("Bắt đầu trích xuất dữ liệu Bonbanh...")
bonbanh_data = []

# Cấu hình thời gian nghỉ (giây)
N = 0.5 

# Hàm wrapper để thêm delay trước khi gọi hàm gốc
def delayed_extract(url):
    # Mỗi thread ngủ N + random(1-3s) trước khi hành động
    # Điều này giúp các thread không gửi request cùng lúc (burst), bonbanh có cơ chế chống burst rất nhạy
    sleep(N + uniform(1, 3))
    return extract_bonbanh_features(url)

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = {executor.submit(delayed_extract, url): url for url in bonbanh_urls}

    for future in as_completed(futures):
        url = futures[future]
        try:
            features = future.result()
            if features:
                print(features)
                bonbanh_data.append(features)
            else:
                print(f"Lỗi với URL Bonbanh {url}")
        except Exception as e:
            print(f"Lỗi với URL Bonbanh {url}: {e}")


df_bonbanh = pd.DataFrame(bonbanh_data)
df_bonbanh.to_csv('MyDrive/raw_bonbanh_car_features.csv', index=False)
print(f"Bonbanh: Đã lưu {len(bonbanh_data)} bản ghi vào datasets/raw_bonbanh_car_features.csv")


## Kết quả
- **Chotot**: Đã lưu dữ liệu vào `datasets/raw_chotot_car_features.csv`, mỗi bản ghi bao gồm tiêu đề, giá, các thông số kỹ thuật, vị trí, người bán và mô tả.
- **Bonbanh**: Đã lưu dữ liệu vào `datasets/raw_bonbanh_car_features.csv`, giữ nguyên tiêu đề gốc và các thuộc tính cốt lõi đã được lọc.
- **Tối ưu hóa**: Nhờ sử dụng đa luồng, thời gian trích xuất giảm đáng kể so với xử lý tuần tự. Với 100 URL, thời gian giảm từ ~150 giây xuống còn khoảng 15-20 giây (tùy thuộc tốc độ mạng và server).